**Technical Note**:
- Install <code>deap</code> in ALMA cluster without root user<br>
    <code>pip install --user deap</code><br>
  Then it is automatically included in CASA environment<br>



## First Run

I use parameters as follow:

Function setting
- Objective function is to minimize the RMS
- using 10 MS from B6
- visweight of the first MS set to 1, the other (9) as free parameter that we want to find. We can try to set all of the visweight as free parameter. 
- visweight is generated as uniform random variable from 0 to 4
- CLEAN parameter:
    - niter          = 500
    - threshold      = "0.035mJy"
    - psfmode        = 'clark'
    - interactive    = False 
    - mask           = "circle[[65.8158364deg, -1.3425182deg], 5arcsec]"
    - imsize         = 1250
    - cell           = "0.04arcsec"
    - phasecenter    = "J2000 04:23:15.800730 -01.20.33.065501"
    - weighting      = 'briggs'
    - robust         = 0.5
    - pbcor          = False
- Region to calculate RMS with imstat: 
    - regionrms      = "annulus[[65.8158364deg, -1.3425182deg], [5arcsec, 15arcsec]]"

GA setting:
- Simple GA steps: selection, crossover, and mutation
- Population size: 10
- Insert 3 best guess at first generation (weight based on 1/rms^2 and all 1 (no weight scale))
- GA iteration/generation: 20
- crossover probability: 0.5
- probability of individu get a mutation: 0.1
- DNA mutation probability: 0.8
- 

Finished < 6 hours  

**Final result**:

- Best of the best:         [0.0, 1.071, 1, 1, 0.8, 0.0, 0.08, 0.05, 0.05] + first MS weight (always) 1 

    with fitness (rms) = 3.1720439437776804e-05

- total evaluation "concat+clean process" : 164 (if the population always change (mutate) it should be 10 x 20=200)


**Comments**:

1. compare to 1/rms^2 scale: [0.55, 0.82, 0.34, 0.28, 0.80, 0.37, 0.08, 0.05, 0.05] + first MS weight (always) 1
    (with rms ~ 3.4uJy after 1000 iteration)
    
    the final result (best of the best) parameter above is a bit weird: we can achieve comparably (lower) rms without including 2nd MS and 7th MS (0.0 in weight)?

2. something weird on 2nd MS (0th in plot below) the weight-value can change from 0 to 3 without changing much of rms. -> divergent case, 

3. we can check the data of 2nd MS, my guess is maybe because it is redundat with the 1st MS that we set to 1, they are from same EB, 1st is as bandpass calibrator, 2nd as phase calib, or vice versa (I don't remember ;) ).

4. From the best result, it seems that CLEAN can achieve the threshold (35uJy) before iter_max reached (500 iterations). So, for next testing we should just set the threshold to 0.0, so all CLEANing process have same number of iteration (stoped because of iter_max)




This plot is statistics of population as function of generation, x-axis is generation/iteration, y-axis is fitness (rms), and statistics shown are minimum (best), average, and maximum.
<img src="stat_fitness.png" alt="stat" style="width: 600px;"/>
---


We can plot the parameter of "best individu" in population as function of generation.
0 - 8 means weight of the 2nd MS to 10th MS. x-axis is generation/iteration, y-axis is weight of each MS.
<img src="best_weight_on_iteration.png" alt="stat" style="width: 600px;"/>






---
## Second Run

for the second run, we
- remove the threshold -> set it to '0mJy' (so it really stop due to niter)
- change the mutation probability
- increase the population also generations (iteration)

Parameter:
+ population: 20
+ generation: 50
+ indpb: 0.2
+ mutpb: 0.5

After running ~ 1 day

Total evalution of "concat+clean": 734

Weird/unexpected result:

+ best weight = [1 (fixed), 0.0, 0.0, 2.6, 0.0, 0.8, 0.0, 0.08, 0.05, 0.05]

+ fitness (rms) = 3.1307943572755903e-05

Many 0's !


Statistic of population as function of generation
<img src="stat2.png" alt="stat" style="width: 600px;"/>

---

---

Best weight of each MS as function of generation
<img src="bestweight2.png" alt="stat" style="width: 600px;"/>

Comments:

- We can try to see the connection (I add intent, because my hypothesis after the first run)

<table style="width:100%">
  <tr>
    <th>Projects</th>
    <th>MS</th>
    <th>Individual RMS</th>
    <th>Weight</th>
    <th>Intent</th>
  </tr>
  <tr>
    <td>2013.1.01241.S</td>
    <td>A002_X9646fb_Xccd.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td> 
    <td>5.8e-05</td>
    <td>1 (fixed)</td>
    <td>Bandpass</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_X9646fb_Xccd.ms.split.cal.clb.field_1.J0423-0120.ms.self3.substracted.ms</td>
    <td>7.8e-05</td> 
    <td>0</td>
    <td>Flux</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_X97db9c_X159b.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td>
    <td>6.4e-05</td> 
    <td>0</td>
    <td>Bandpass</td>
  </tr>
  <tr>
    <td>2012.1.00304.S</td>
    <td>A002_X95b353_X6f7.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td>
    <td>1.0e-04</td> 
    <td>2.6</td>
    <td>Bandpass</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_X9630c0_X81b.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td>
    <td>1.1e-04</td> 
    <td>0</td>
    <td>Bandpass</td>
  </tr>
  <tr>
    <td>2013.1.00916.S</td>
    <td>A002_Xa916fc_X6d6b.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td>
    <td>6.5e-05</td> 
    <td>0.8</td>
    <td>Bandpass and Phase</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_Xa916fc_X6d6b.ms.split.cal.clb.field_1.J0423-0120.ms.self3.substracted.ms</td>
    <td>9.5e-05</td> 
    <td>0</td>
    <td>Flux</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_Xaaa05f_X15bb.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td>
    <td>2e-04</td> 
    <td>0.08</td>
    <td>Bandpass and Delay</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_Xaaa05f_X1af7.ms.split.cal.clb.field_0.J0423-0120.ms.self3.substracted.ms</td>
    <td>2.6e-04</td> 
    <td>0.05</td>
    <td>Bandpass and Delay</td>
  </tr>
  <tr>
    <td></td>
    <td>A002_Xaaa05f_X1af7.ms.split.cal.clb.field_1.J0423-0120.ms.self3.substracted.ms</td>
    <td>2.6e-04</td> 
    <td>0.05</td>
    <td>Flux</td>
  </tr>
</table>

From the table above:
- 1st MS of each project get "highest" weight, and the other can be "ignored". hmmm...